In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_capacity_by_dc_rop
USING DELTA
AS

SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 AS calday,
    dc_id,
    CASE 
        WHEN DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) = DATE('2024-02-03') THEN 1000000000
        ELSE capacity
    END AS capacity,
    last_update_date
    FROM {catalog_name}.udp_wcm_silver_rop.dc_demand_supply_config
    QUALIFY ROW_NUMBER() OVER (PARTITION BY dc_id ORDER BY last_update_date DESC, proc_date DESC) =1

""")
